## Define Slice Topology

### Imports 

In [ ]:
# Import libraries
%run ../setup/include/include_libraries.py
# load general variables
%run ../setup/config/load_variables.py
# load topology variables
%run topology_variables.ipynb

from state import *
import write_value

### Slice Topology

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    [site1,site2,site3] = fablib.get_random_sites(count=3)

    
    if (h1_site == '' and s1_site == ''):
        hs1_site = site1
        write_value.write_value_to_file('topology_variables.ipynb', 'h1_site', hs1_site)
        write_value.write_value_to_file('topology_variables.ipynb', 's1_site', hs1_site)
    if (h2_site =='' and s2_site == ''):
        hs2_site = site2
        write_value.write_value_to_file('topology_variables.ipynb', 'h2_site', hs2_site)
        write_value.write_value_to_file('topology_variables.ipynb', 's2_site', hs2_site)
        
    if (h3_site =='' and s3_site == ''):
        hs3_site = site1
        write_value.write_value_to_file('topology_variables.ipynb', 'h3_site', hs3_site)
        write_value.write_value_to_file('topology_variables.ipynb', 's3_site', hs3_site)
        
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=hs1_site,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1_iface_local = s1.add_component(model=nic_model, name=s1_nic1_name).get_interfaces()[0]
    s1_iface_to_s2 = s1.add_component(model=nic_model, name=s1_nic2_name).get_interfaces()[0]
    s1_iface_to_s3 = s1.add_component(model=nic_model, name=s1_nic3_name).get_interfaces()[0]

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site2,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2_iface_local = s2.add_component(model=nic_model, name=s2_nic1_name).get_interfaces()[0]
    s2_iface_to_s1 = s2.add_component(model=nic_model, name=s2_nic2_name).get_interfaces()[0]
    s2_iface_to_s3 = s2.add_component(model=nic_model, name=s2_nic3_name).get_interfaces()[0]
    
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site3,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3_iface_local = s3.add_component(model=nic_model, name=s3_nic1_name).get_interfaces()[0]
    s3_iface_to_s1 = s3.add_component(model=nic_model, name=s3_nic2_name).get_interfaces()[0]
    s3_iface_to_s2 = s3.add_component(model=nic_model, name=s3_nic3_name).get_interfaces()[0]    
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model=nic_model, name=h1_nic_name).get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model=nic_model, name=h2_nic_name).get_interfaces()[0]
    
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site3, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h3_iface = h3.add_component(model=nic_model, name=h3_nic_name).get_interfaces()[0]
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])
    
    success = True
    
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

### Set State to "SLICE_DEFINED" 

In [ ]:
if success:
    state_data = {'slice_name': slice_name, 'state': ['SLICE_DEFINED']}
    set_state_local(state_data)
else:
    print ('State is not updated due to error')